In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
872,I have done quite a few reviews on IMDb and th...,negative
798,"The Monkees, surprisingly, are a big favorite ...",positive
255,"Well, not much really to say about this. But i...",positive
524,This movie isn't about football at all. It's a...,negative
172,When HOPPITY GOES TO TOWN he discovers nothing...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
872,done quite review imdb film unique never saw e...,negative
798,monkees surprisingly big favorite mine yes mig...,positive
255,well much really say really good good job soft...,positive
524,movie football jesus god kind sappy sanctimoni...,negative
172,hoppity go town discovers nothing bad news lit...,positive


In [7]:
df['sentiment'].value_counts()

sentiment
negative    253
positive    247
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
872,done quite review imdb film unique never saw e...,0
798,monkees surprisingly big favorite mine yes mig...,1
255,well much really say really good good job soft...,1
524,movie football jesus god kind sappy sanctimoni...,0
172,hoppity go town discovers nothing bad news lit...,1


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [18]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [27]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [28]:
import dagshub

mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
dagshub.init(repo_owner=os.environ['DAGSHUB_OWNER'], repo_name=os.environ['DAGSHUB_REPO'], mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2026-01-14 22:52:36,124 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/shabbirbilal02/sentiment-analysis-mlops-platform "HTTP/1.1 200 OK"


Initialized MLflow to track repo "shabbirbilal02/sentiment-analysis-mlops-platform"

2026-01-14 22:52:36,130 - INFO - Initialized MLflow to track repo "shabbirbilal02/sentiment-analysis-mlops-platform"


Repository shabbirbilal02/sentiment-analysis-mlops-platform initialized!

2026-01-14 22:52:36,135 - INFO - Repository shabbirbilal02/sentiment-analysis-mlops-platform initialized!


<Experiment: artifact_location='mlflow-artifacts:/0b86d6e6d2e94441b78449d6063b60d2', creation_time=1768423624517, experiment_id='0', last_update_time=1768423624517, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [29]:
import mlflow
import logging
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-14 22:53:25,470 - INFO - Starting MLflow run...
2026-01-14 22:53:25,872 - INFO - Logging preprocessing parameters...
2026-01-14 22:53:27,059 - INFO - Initializing Logistic Regression model...
2026-01-14 22:53:27,059 - INFO - Fitting the model...
2026-01-14 22:53:27,099 - INFO - Model training complete.
2026-01-14 22:53:27,103 - INFO - Logging model parameters...
2026-01-14 22:53:27,411 - INFO - Making predictions...
2026-01-14 22:53:27,411 - INFO - Calculating evaluation metrics...
2026-01-14 22:53:27,427 - INFO - Logging evaluation metrics...
2026-01-14 22:53:28,976 - INFO - Saving and logging the model...
2026/01/14 22:53:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026-01-14 22:53:44,362 - INFO - Model training and logging completed in 18.49 seconds.
2026-01-14 22:53:44,362 - INFO - Accuracy: 0.69
2026-01-14 22:53:44,362 - INFO - Precision: 0.7317073170731707
2026-01-14 22:53:44,362 - INFO - Recall: 0.6
2026-01-14 22:53:44,362 - 

🏃 View run masked-turtle-655 at: https://dagshub.com/shabbirbilal02/sentiment-analysis-mlops-platform.mlflow/#/experiments/0/runs/b185e9d5238a459f91e14f78e5e43ff1
🧪 View experiment at: https://dagshub.com/shabbirbilal02/sentiment-analysis-mlops-platform.mlflow/#/experiments/0
